In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# The Python magic
import IPython
import signal

def interrupted(_interrupted=[False], _default=[None]):
    if _default[0] is None or signal.getsignal(signal.SIGINT) == _default[0]:
        _interrupted[0] = False
        def handle(signal, frame):
            if _interrupted[0] and _default[0] is not None:
                _default[0](signal, frame)
            print('Interrupt!')
            _interrupted[0] = True
        _default[0] = signal.signal(signal.SIGINT, handle)
    return _interrupted[0]

def enumerate_cycle(g):
    epoch = 0
    while True:
        for i,x in enumerate(g):
            yield (epoch,i), x
        epoch = epoch + 1

In [5]:
# Loading the dataset
pd.set_option('display.max_columns', 500)

diabetic = pd.read_csv('diabetes/diabetic_data_original.csv')
diabetic

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,MC,?,51,0,16,0,0,0,250.13,291,458,9,None,>8,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,MC,?,33,3,18,0,0,1,560,276,787,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,MC,?,53,0,9,1,0,0,38,590,296,13,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,MC,Surgery-General,45,2,21,0,0,1,996,285,998,9,None,None,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO


In [40]:
np.unique(diabetic['patient_nbr'].value_counts().values, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 28, 40]),
 array([54745, 10434,  3328,  1421,   717,   346,   207,   111,    70,
           42,    20,    19,    14,     5,     9,     4,     3,     6,
            3,     6,     1,     2,     3,     1,     1]))

In [41]:
54745/len(diabetic)

0.5379498064186466

In [45]:
(54745+10434)/len(diabetic)

0.6404791384155808

In [4]:
patient_nbr_counts = diabetic['patient_nbr'].value_counts()
df_diabetic = diabetic[diabetic['patient_nbr'].isin(patient_nbr_counts.index[patient_nbr_counts.gt(1)])].sort_values(['patient_nbr', 'encounter_id'])
pd.set_option('display.max_rows', 25)
print(df_diabetic[['encounter_id', 'patient_nbr', 'readmitted']])

        encounter_id  patient_nbr readmitted
4267        24437208          135        <30
4780        26264286          135        >30
1164         8380170         1152        >30
5953        30180318         1152        >30
14180       55533660         1152        >30
...              ...          ...        ...
99544      414418886    188634893         NO
93050      330256946    188970179        <30
101595     441488168    188970179         NO
90933      302575430    189257846         NO
98191      397823084    189257846        >30

[47021 rows x 3 columns]


Patients that have been readmitted multiple times

In [7]:
df_diabetic = df_diabetic.set_index(df_diabetic['encounter_id'])
df_diabetic['len_next_stay'] = pd.Series()
for patient in df_diabetic['patient_nbr'].unique():
    encounter_ids = list(df_diabetic[df_diabetic['patient_nbr'] == patient]['encounter_id'])
    readmitted = df_diabetic.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    lengths_of_stay = df_diabetic.loc[encounter_ids, 'time_in_hospital']
    df_diabetic.loc[encounter_ids[:-1], ['len_next_stay']] = lengths_of_stay.values[1:]

df_readmitted_only = df_diabetic.copy()
df_readmitted_only = df_readmitted_only[np.isfinite(df_readmitted_only['len_next_stay'])]

In [10]:
np.unique(df_readmitted_only['patient_nbr'].value_counts().values, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 27, 39]),
 array([10434,  3328,  1421,   717,   346,   207,   111,    70,    42,
           20,    19,    14,     5,     9,     4,     3,     6,     3,
            6,     1,     2,     3,     1,     1]))

Removing the patients that have been not readmitted more than once or have been readmitted after NO readmission outcome

In [26]:
multiple_not_readmitted = []
for patient in df_diabetic['patient_nbr'].unique():
    encounter_ids = list(df_diabetic[df_diabetic['patient_nbr'] == patient]['encounter_id'])
    readmitted = df_diabetic.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    if 'NO' in readmitted and readmitted['NO'] > 1:
        multiple_not_readmitted.append(patient)

len(multiple_not_readmitted)

44

In [29]:
df_readmitted_only[df_readmitted_only['readmitted'] == 'NO']

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,len_next_stay
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90671322,90671322,1189206,AfricanAmerican,Male,[80-90),?,2,3,1,3,MC,InternalMedicine,31,0,9,0,0,0,436,496,486,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,9.0
82617192,82617192,1305612,AfricanAmerican,Female,[70-80),?,2,3,1,6,MC,InternalMedicine,56,1,26,0,0,1,403,250.82,707,5,None,>7,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO,6.0
90400272,90400272,1405557,AfricanAmerican,Female,[70-80),?,2,1,1,9,MC,Nephrology,43,4,13,0,0,0,440,707,403,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,14.0
93566430,93566430,3195774,Caucasian,Male,[70-80),?,2,1,4,9,HM,Nephrology,44,4,20,0,0,0,410,428,403,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,13.0
91783026,91783026,16697187,Caucasian,Female,[70-80),?,2,1,1,6,HM,PhysicalMedicineandRehabilitation,45,1,20,0,0,0,715,285,401,5,None,None,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293778884,293778884,186400166,Caucasian,Male,[80-90),?,1,3,7,3,MC,?,19,0,11,0,0,1,348,294,E939,8,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,2.0
377345516,377345516,187298114,Caucasian,Male,[80-90),?,1,1,7,3,MC,?,50,0,16,0,0,4,428,404,425,9,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,2.0
337315430,337315430,187920311,Caucasian,Female,[60-70),?,3,6,1,11,PO,?,37,1,16,0,0,1,V57,V54,V54,9,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO,2.0


In [27]:
to_exclude = []
for patient in df_readmitted_only['patient_nbr'].unique():
    encounter_ids = list(df_readmitted_only[df_readmitted_only['patient_nbr'] == patient]['encounter_id'])
    readmitted = df_readmitted_only.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    if 'NO' in readmitted:
        to_exclude.append(patient)

to_exclude

[1189206,
 1305612,
 1405557,
 3195774,
 16697187,
 17079237,
 18627624,
 23778297,
 27956088,
 29108259,
 29324601,
 29368215,
 29412864,
 30184254,
 30325419,
 30325446,
 30526470,
 30897927,
 31355928,
 31365891,
 31652982,
 31665015,
 31665987,
 32011137,
 32236452,
 32263560,
 32311764,
 32314608,
 32337963,
 32565870,
 32673303,
 33108129,
 33230016,
 33379713,
 33454008,
 33581808,
 33845625,
 33896214,
 33946560,
 34248249,
 34301979,
 34507611,
 35047287,
 35052723,
 35510715,
 35816535,
 35967501,
 36161775,
 36216972,
 36240570,
 36851895,
 37010466,
 37255248,
 37459422,
 37592973,
 38045853,
 38056068,
 38074284,
 38075310,
 38082618,
 38147922,
 38225853,
 38264805,
 38502261,
 38515230,
 38657925,
 38666502,
 38768337,
 38774187,
 38799234,
 38812941,
 39125133,
 39433527,
 39602736,
 39626514,
 39740490,
 39783186,
 39791142,
 39857706,
 39911454,
 39923199,
 40325778,
 40373325,
 40395537,
 40442940,
 40449735,
 40475232,
 40520277,
 40534695,
 40623579,
 40668219,
 40

In [30]:
len(to_exclude)

489

(the discrepancy between the above 490 rows and `to_exclude=489` is related to that one patient is still not readmitted more than once)

In [31]:
df_filtered = df_readmitted_only.drop(df_readmitted_only[df_readmitted_only['readmitted'] == 'NO'].index, axis=0)
df_filtered

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,len_next_stay
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24437208,24437208,135,Caucasian,Female,[50-60),?,2,1,1,8,?,Cardiology,77,6,33,0,0,0,401,997,560,8,None,None,Steady,No,No,No,No,No,No,Down,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,<30,3.0
8380170,8380170,1152,AfricanAmerican,Female,[50-60),?,1,1,7,6,?,Hematology/Oncology,43,2,13,0,0,1,282,250.01,?,2,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,6.0
30180318,30180318,1152,AfricanAmerican,Female,[50-60),?,1,1,7,6,?,Hematology/Oncology,45,4,15,0,0,2,282,794,250,6,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30,10.0
55533660,55533660,1152,AfricanAmerican,Female,[60-70),?,1,1,7,10,?,Hematology/Oncology,54,2,19,0,0,1,282,276,428,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,8.0
80742510,80742510,1152,AfricanAmerican,Female,[60-70),?,1,1,7,8,?,?,30,1,16,0,0,1,282,250,?,2,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322610720,322610720,187298114,Caucasian,Male,[80-90),?,1,1,7,2,MC,?,13,0,16,0,0,3,404,585,428,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,<30,4.0
363787034,363787034,187298114,Caucasian,Male,[80-90),?,1,6,7,4,MC,Resident,71,0,18,0,0,2,276,584,427,9,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,<30,3.0
344902604,344902604,188284883,Other,Female,[80-90),?,1,1,7,1,MC,?,61,2,9,0,0,0,531,211,562,5,None,None,No,No,No,No,No,No,Steady,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,>30,2.0


Number of patients:

In [34]:
len(np.unique(df_filtered['patient_nbr']))

16515

Sequence lengths:

In [32]:
np.unique(df_filtered['patient_nbr'].value_counts().values, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 27, 39]),
 array([10280,  3276,  1403,   698,   338,   207,   110,    65,    42,
           21,    18,    14,     5,     8,     4,     3,     6,     3,
            6,     1,     2,     3,     1,     1]))

In [39]:
10280/16515

0.6224644262791402

## Handling missing values

In [35]:
for column in list(df_diabetic.columns.values):
    df_filtered[column] = np.where(df_filtered[column] == '?', np.nan, df_filtered[column])
df_filtered

/Users/kamilestankeviciute/anaconda3/envs/datasci/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,len_next_stay
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24437208,24437208.0,135.0,Caucasian,Female,[50-60),NaN,2.0,1.0,1.0,8.0,NaN,Cardiology,77.0,6.0,33.0,0.0,0.0,0.0,401,997,560,8.0,None,None,Steady,No,No,No,No,No,No,Down,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,<30,3.0
8380170,8380170.0,1152.0,AfricanAmerican,Female,[50-60),NaN,1.0,1.0,7.0,6.0,NaN,Hematology/Oncology,43.0,2.0,13.0,0.0,0.0,1.0,282,250.01,NaN,2.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,6.0
30180318,30180318.0,1152.0,AfricanAmerican,Female,[50-60),NaN,1.0,1.0,7.0,6.0,NaN,Hematology/Oncology,45.0,4.0,15.0,0.0,0.0,2.0,282,794,250,6.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30,10.0
55533660,55533660.0,1152.0,AfricanAmerican,Female,[60-70),NaN,1.0,1.0,7.0,10.0,NaN,Hematology/Oncology,54.0,2.0,19.0,0.0,0.0,1.0,282,276,428,9.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,8.0
80742510,80742510.0,1152.0,AfricanAmerican,Female,[60-70),NaN,1.0,1.0,7.0,8.0,NaN,NaN,30.0,1.0,16.0,0.0,0.0,1.0,282,250,NaN,2.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322610720,322610720.0,187298114.0,Caucasian,Male,[80-90),NaN,1.0,1.0,7.0,2.0,MC,NaN,13.0,0.0,16.0,0.0,0.0,3.0,404,585,428,9.0,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,<30,4.0
363787034,363787034.0,187298114.0,Caucasian,Male,[80-90),NaN,1.0,6.0,7.0,4.0,MC,Resident,71.0,0.0,18.0,0.0,0.0,2.0,276,584,427,9.0,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,<30,3.0
344902604,344902604.0,188284883.0,Other,Female,[80-90),NaN,1.0,1.0,7.0,1.0,MC,NaN,61.0,2.0,9.0,0.0,0.0,0.0,531,211,562,5.0,None,None,No,No,No,No,No,No,Steady,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,>30,2.0


## Generating encounter sequences